In [1]:
import os

In [2]:
%pwd

'd:\\IBSS-with-mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config #give all other ingestion here 

In [7]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-30 06:36:52,407: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-30 06:36:52,415: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-30 06:36:52,423: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-30 06:36:52,423: INFO: common: created directory at: artifacts]
[2024-01-30 06:36:52,431: INFO: common: created directory at: artifacts/data_ingestion]


[2024-01-30 06:36:56,864: INFO: 2366278714: artifacts/data_ingestion/default_credit.zip download! with following info: 
Connection: close
Content-Length: 968884
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "25b2567802bbb8183e22df68c212c789d094cca70cfa3d206b204462d1c3d931"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EAB6:6666:C13F1D:EE6112:65B84B1C
Accept-Ranges: bytes
Date: Tue, 30 Jan 2024 01:06:57 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10227-MAA
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1706576818.775860,VS0,VE2
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 2c6df959aa174b1bfe7b66edc8be8483ffc2764a
Expires: Tue, 30 Jan 2024 01:11:57 GMT
Source-Age: 149

]
